# Checkpoint Three: Cleaning Data

Now you are ready to clean your data. Before starting coding, provide the link to your dataset below.

My dataset:

Import the necessary libraries and create your dataframe(s).

In [215]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style

#Setting up dataframes
titles_df = pd.read_csv("titles-to-clean.csv")
credits_df = pd.read_csv("credits-to-clean.csv")

## Missing Data

Test your dataset for missing data and handle it as needed. Make notes in the form of code comments as to your thought process.

In [216]:
# For the Credits database, characters column contains a significant number of missing values. I do not plan on using this for my purposes anyway so I will be removing 
# the entire column
credits_df = credits_df.drop("character", axis=1)
credits_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66393 entries, 0 to 66392
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   person_id  66393 non-null  int64 
 1   id         66393 non-null  object
 2   name       66393 non-null  object
 3   role       66393 non-null  object
dtypes: int64(1), object(3)
memory usage: 2.0+ MB


The above information displays that the Character column has been removed from the Credits DataFrame. 

In [217]:
# I am going to clean up any row that is missing both IMDb and TMDb score values since most of my business questions rely on these data points.
titles_df = titles_df.dropna(subset=['imdb_score', 'tmdb_score'], how='all')
titles_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3188 entries, 0 to 3293
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    3188 non-null   object 
 1   title                 3188 non-null   object 
 2   type                  3188 non-null   object 
 3   description           3185 non-null   object 
 4   release_year          3188 non-null   int64  
 5   age_certification     2064 non-null   object 
 6   runtime               3188 non-null   int64  
 7   genres                3188 non-null   object 
 8   production_countries  3188 non-null   object 
 9   seasons               730 non-null    float64
 10  imdb_id               2943 non-null   object 
 11  imdb_score            2922 non-null   float64
 12  imdb_votes            2911 non-null   float64
 13  tmdb_popularity       3168 non-null   float64
 14  tmdb_score            3026 non-null   float64
dtypes: float64(5), int64(2), o

The information above displays that 106 rows were removed from the Titles DataFrame. 

In [218]:
# Next I am dropping the Description column from Titles. This column contains null values and plus I will not be using this data.
titles_df = titles_df.drop("description", axis=1)
titles_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3188 entries, 0 to 3293
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    3188 non-null   object 
 1   title                 3188 non-null   object 
 2   type                  3188 non-null   object 
 3   release_year          3188 non-null   int64  
 4   age_certification     2064 non-null   object 
 5   runtime               3188 non-null   int64  
 6   genres                3188 non-null   object 
 7   production_countries  3188 non-null   object 
 8   seasons               730 non-null    float64
 9   imdb_id               2943 non-null   object 
 10  imdb_score            2922 non-null   float64
 11  imdb_votes            2911 non-null   float64
 12  tmdb_popularity       3168 non-null   float64
 13  tmdb_score            3026 non-null   float64
dtypes: float64(5), int64(2), object(7)
memory usage: 373.6+ KB


The Description column has successfully been removed from the Titles DataFrame.

In [219]:
# There are many null values in the Age Certification column. I do not want to drop these rows, so I will be replacing the NaN value with "Unrated". This way I will 
# still be able to perform analysis with this information and I will simply be creating a new category type for this column.
titles_df["age_certification"] = titles_df["age_certification"].fillna("Unrated")
titles_df["age_certification"].isna().sum()
titles_df["age_certification"].value_counts()

age_certification
Unrated    1124
R           595
PG-13       469
TV-MA       311
PG          297
TV-14       143
G            83
TV-PG        83
TV-Y7        41
TV-G         20
TV-Y         15
NC-17         7
Name: count, dtype: int64

There are no longer any missing values in the Age Certification column and the null values have now been replaced with Unrated.

In [220]:
# There are many missing values in the Seasons column, but I am wondering if most of these null values are because it is a Movie Title and not a TV Show. 
# I am going to check to see how many null values are present if the title is a TV Show.
missing_TV_season = titles_df.groupby("type")["seasons"].apply(lambda x : x.isnull().sum())
missing_TV_season

type
MOVIE    2458
SHOW        0
Name: seasons, dtype: int64

The above output shows that the missing data in the Seasons column is in fact because that title is a Movie and not a TV Show. I will not be removing or changing 
these values in this case. If I want to use the seasons column, I will filter it and null values will no longer be an issue.

In [221]:
# I want to take a look at how many more missing values there are besides in the Seasons column.
titles_df.isna().sum()

id                         0
title                      0
type                       0
release_year               0
age_certification          0
runtime                    0
genres                     0
production_countries       0
seasons                 2458
imdb_id                  245
imdb_score               266
imdb_votes               277
tmdb_popularity           20
tmdb_score               162
dtype: int64

The remaining missing values are in the IMDb and TMDb categories. I do not want to make any changes to these currently, because I do not want to skew any results
by replacing the null with a new value. I will leave them as is for now and decide how to proceed with them during my manipulation phase.

## Irregular Data

Detect outliers in your dataset and handle them as needed. Use code comments to make notes about your thought process.

In [222]:
# I can see from a scatter chart in my EDA that there are several titles that have IMDb votes that are over 1,000,000. I want to see how many exactly there are.
title_count = []
for i in ("title"):
    title_count = (titles_df["imdb_votes"] > 1000000).sum()
print(title_count)

19


In [223]:
titles_df["imdb_votes"].describe()
# There is a pretty wide spread of vote counts from IMDb, but I do feel like more popular titles will have more votes. During my EDA I do think higher vote counts
# correlated with higher IMDb scores.

count    2.911000e+03
mean     6.461835e+04
std      1.754617e+05
min      5.000000e+00
25%      9.245000e+02
50%      8.071000e+03
75%      4.764500e+04
max      2.555504e+06
Name: imdb_votes, dtype: float64

I have decided not to remove these outliers. I want to investigate popularity of titles and a title having an excess of votes just tells me that it is in fact popular. I am not sure if I will be looking at data based on the number of votes moving forward, so I do not want to remove these titles when their score info will be relevant to my research and vote count may not be. 

## Unnecessary Data

Look for the different types of unnecessary data in your dataset and address it as needed. Make sure to use code comments to illustrate your thought process.

I have already removed several columns of unnecessary data. In the Titles DataFrame I have removed the Description column and in the Credits DataFrame I have removed the Characters column. I will now be taking a look at the DataFrames and seeing if there are any rows or more columns that I would like to remove.

In [224]:
# I am not going to be using any actor data from the Credits DataFrame, so I will be removing all of those rows.
credits_to_keep = credits_df["role"] == "DIRECTOR"
credits_df = credits_df[credits_to_keep]


In [225]:
credits_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2774 entries, 22 to 66392
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   person_id  2774 non-null   int64 
 1   id         2774 non-null   object
 2   name       2774 non-null   object
 3   role       2774 non-null   object
dtypes: int64(1), object(3)
memory usage: 108.4+ KB


The Credits DataFrame has now been reduced to 2774 rows from 66,393.

In [226]:
# I do not have a use for the IMDb ID column in the Titles DataFrame, so I will be dropping this column.
titles_df = titles_df.drop("imdb_id", axis=1)
titles_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3188 entries, 0 to 3293
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    3188 non-null   object 
 1   title                 3188 non-null   object 
 2   type                  3188 non-null   object 
 3   release_year          3188 non-null   int64  
 4   age_certification     3188 non-null   object 
 5   runtime               3188 non-null   int64  
 6   genres                3188 non-null   object 
 7   production_countries  3188 non-null   object 
 8   seasons               730 non-null    float64
 9   imdb_score            2922 non-null   float64
 10  imdb_votes            2911 non-null   float64
 11  tmdb_popularity       3168 non-null   float64
 12  tmdb_score            3026 non-null   float64
dtypes: float64(5), int64(2), object(6)
memory usage: 348.7+ KB


IMDb ID column has been successfully dropped from the Titles DataFrame.

## Inconsistent Data

Check for inconsistent data and address any that arises. As always, use code comments to illustrate your thought process.

In [227]:
# The production countries are all listed as 2 letter codes, but I would like to see what the full names of these countries are. I will be replacing the codes with 
# their full names, but first I will be replacing the [] values with "Unknown".
titles_df["production_countries"] = titles_df["production_countries"].replace("[]", "Unknown")

# Next I need to remove the brackets and quotes from each of the country codes.
titles_df["production_countries"] = titles_df["production_countries"].str.replace("[","")
titles_df["production_countries"] = titles_df["production_countries"].str.replace("]","")
titles_df["production_countries"] = titles_df["production_countries"].str.replace("'","")

#There are so many different country codes that I will be using the country_converter library to convert the codes to full country names.
import country_converter as coco       
cc = coco.CountryConverter()

# There are some production countries that list multiple countries per title, so I will need to split these up and convert each code individually.
def convert_comma_separated_codes(code_string):
    codes_list = [code.strip() for code in code_string.split(',')]
    
    # Convert the list of codes using country_converter,'not found' codes will be changed to 'Unknown'.
    names_list = cc.convert(names=codes_list, to='name_short', not_found='Unknown')
    
    if isinstance(names_list, str):
        names_list = [names_list]
        
    # Join the converted names back into a single comma-separated string
    return ', '.join(names_list)

titles_df['production_countries'] = titles_df['production_countries'].apply(convert_comma_separated_codes)
titles_df["production_countries"].value_counts()

SU not found in ISO2
SU not found in ISO2
XG not found in ISO2
Unknown not found in regex
Unknown not found in regex
Unknown not found in regex
SU not found in ISO2
SU not found in ISO2
XC not found in ISO2
XC not found in ISO2
SU not found in ISO2
SU not found in ISO2
XC not found in ISO2
SU not found in ISO2
SU not found in ISO2
SU not found in ISO2
SU not found in ISO2
SU not found in ISO2
Unknown not found in regex
Unknown not found in regex
Unknown not found in regex
Unknown not found in regex
Unknown not found in regex
XX not found in ISO2
XX not found in ISO2
Unknown not found in regex
Unknown not found in regex
Unknown not found in regex
XX not found in ISO2
XX not found in ISO2
Unknown not found in regex
Unknown not found in regex
Unknown not found in regex
Unknown not found in regex
Unknown not found in regex
XX not found in ISO2
Unknown not found in regex
XX not found in ISO2
Unknown not found in regex
Unknown not found in regex
Unknown not found in regex
Unknown not found i

production_countries
United States                    2036
United Kingdom                    144
Japan                             128
Unknown                            85
United Kingdom, United States      64
                                 ... 
Spain, Argentina                    1
France, Spain                       1
Belgium, France, Guatemala          1
Türkiye                             1
Panama                              1
Name: count, Length: 264, dtype: int64

In [228]:
titles_df["production_countries"].value_counts().sum()

np.int64(3188)

All production countries have all been changed to their full country names, with codes that were not found in the country converter now listed as Unknown.

In [229]:
# The genre column contains several different genres per title and they are all listed with brackets and quotes. I will be removing these.
titles_df["genres"] = titles_df["genres"].str.replace("[","")
titles_df["genres"] = titles_df["genres"].str.replace("]","")
titles_df["genres"] = titles_df["genres"].str.replace("'","")   
titles_df["genres"].value_counts()

genres
documentation                      328
drama                              180
comedy                             178
comedy, documentation               60
comedy, drama                       50
                                  ... 
documentation, music, history        1
drama, family, comedy                1
animation, action, fantasy           1
thriller, family, comedy, drama      1
thriller, romance, crime             1
Name: count, Length: 1127, dtype: int64

All genres have now had the brackets and quotations removed. I will now be exporting out my new cleaned CSVs.

In [230]:
titles_df.to_csv("clean_titles.csv")
credits_df.to_csv("clean_credits.csv")

## Summarize Your Results

Make note of your answers to the following questions.

1. Did you find all four types of dirty data in your dataset?

I do not believe I truly found any Irregular data. I did do a check on some data that could be an outliers if I was using this data set in a different way, but for my purposes I did not need to removing any data that would be considered an outlier. I did find many missing values, I removed unnecessary data columns and rows, and I changed formatting in a couple of columns.

2. Did the process of cleaning your data give you new insights into your dataset?

Now that I have cleaned these data sets, I do feel much more acquanited with the data and it helped me make some decisions in regards to how I would like to manipulate and what insights I will be planning on looking at.

3. Is there anything you would like to make note of when it comes to manipulating the data and making visualizations?

I feel like I have a pretty decent plan on how I would like to manipulate and visualize this data set. I will be mostly looking at different factors that may have influenced higher or lower scoring from audience rating sites.